### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
# !pip install langchain

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
# !pip install langchain_groq

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022372B1C040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022372B1CBB0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [5]:
# !pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage
# INstruction for who is giving the instruction
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content="Hello, how are you? in French is:\n\n**Bonjour, comment allez-vous ?** \n\n You can also use the more informal:\n\n**Salut, comment vas-tu?** \n\n\nLet me know if you have any other phrases you'd like translated!\n", response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 26, 'total_tokens': 86, 'completion_time': 0.123698646, 'prompt_time': 0.002617285, 'queue_time': None, 'total_time': 0.126315931}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f83288c2-bc55-4ec3-a567-d30f185daad0-0', usage_metadata={'input_tokens': 26, 'output_tokens': 60, 'total_tokens': 86})

In [8]:
from langchain_core.output_parsers import StrOutputParser
# Convert it in the string display
parser=StrOutputParser()
parser.invoke(result)

"Hello, how are you? in French is:\n\n**Bonjour, comment allez-vous ?** \n\n You can also use the more informal:\n\n**Salut, comment vas-tu?** \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [9]:
### Using LCEL- chain the components
chain=model|parser # This is creating a chain of models
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [16]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [18]:
result # It has converted it into list of messages 

ChatPromptValue(messages=[SystemMessage(content='Trnaslate the following into French:'), HumanMessage(content='Hello')])

In [17]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:'),
 HumanMessage(content='Hello')]

In [15]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Japanese","text":"I am Monkey D. Luffy and I am going to be the king of pirates"})

"私はモンキー・D・ルフィであり、海賊王になるんだ！\n\n(Watashi wa Monkey D. Rurifī de ari, kaizoku ō ni naru n da!) \n\n\nLet me know if you'd like me to translate anything else!\n"

In [19]:
# !pip install streamlit